<a href="https://colab.research.google.com/github/mynameislllyt/API_Experiment/blob/main/baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ============ 1. 读数据 + 拆 benign-only ============
def load_csv_expanded(path):
    df = pd.read_csv(path)
    seq_cols = [c for c in df.columns if c.startswith("t_")]
    seqs = df[seq_cols].values.astype(int)      # shape: [N, 100]
    labels = df["malware"].values.astype(int)   # 1=malware, 0=benign
    return seqs, labels

def split_benign_only(seqs, labels, seed=42):
    benign = seqs[labels == 0]
    malware = seqs[labels == 1]

    rng = np.random.default_rng(seed)
    idx = rng.permutation(len(benign))

    n = len(benign)
    n_train = int(0.7*n)
    n_val   = int(0.1*n)

    train = benign[idx[:n_train]]
    val   = benign[idx[n_train:n_train+n_val]]
    test_benign = benign[idx[n_train+n_val:]]
    test_malware = malware
    return train, val, test_benign, test_malware

# ============ 2. 滑动窗口 ============
def make_windows(seqs, window_size=10):
    X, y = [], []
    for s in seqs:
        s = s.tolist()
        for i in range(len(s) - window_size):
            X.append(s[i:i+window_size])
            y.append(s[i+window_size])
    return np.array(X, dtype=int), np.array(y, dtype=int)

# ============ 3. Dataset ============
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ============ 4. LSTM LM ============
class LSTMLM(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim, num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        e = self.emb(x)        # [B, W, E]
        o, _ = self.lstm(e)    # [B, W, H]
        last = o[:, -1, :]
        return self.fc(last)   # [B, V]

# ============ 5. 训练/验证 ============
def eval_loss(model, loader, device="cuda"):
    model.eval()
    crit = nn.CrossEntropyLoss(reduction="sum")
    total = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            total += crit(model(Xb), yb).item()
    return total / len(loader.dataset)

def train_model(model, train_loader, val_loader, epochs=40, lr=1e-3, device="cuda"):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()

    best_val, best_state = 1e9, None
    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for Xb, yb in tqdm(train_loader, desc=f"Epoch {ep}"):
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = crit(model(Xb), yb)
            loss.backward()
            opt.step()
            total += loss.item() * Xb.size(0)

        val_loss = eval_loss(model, val_loader, device)
        print(f"ep{ep}: train={total/len(train_loader.dataset):.4f}, val={val_loss:.4f}")
        if val_loss < best_val:
            best_val = val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}

    model.load_state_dict(best_state)
    return model

# ============ 6. 序列 NLL 异常分数 ============
def sequence_scores_nll(model, seqs, window_size=10, device="cuda"):#score 越大 = 越不符合 benign 模式 = 越可疑
    crit = nn.CrossEntropyLoss(reduction="none")
    model.eval()
    scores = []

    with torch.no_grad():
        for s in seqs:
            X, y = make_windows([s], window_size)
            X = torch.tensor(X, dtype=torch.long).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)
            nll = crit(model(X), y)
            scores.append(nll.mean().item())
    return np.array(scores)

def pick_threshold(val_scores, q=0.99):#在验证集上，大约 99% 的 benign score 都 低于 这个阈值
    return float(np.quantile(val_scores, q))

def evaluate(th, benign_scores, malware_scores):
    y_true = np.array([0]*len(benign_scores) + [1]*len(malware_scores))
    y_pred = np.array(
        [1 if s>th else 0 for s in benign_scores] +
        [1 if s>th else 0 for s in malware_scores]
    )
    tp = ((y_true==1)&(y_pred==1)).sum()
    tn = ((y_true==0)&(y_pred==0)).sum()
    fp = ((y_true==0)&(y_pred==1)).sum()
    fn = ((y_true==1)&(y_pred==0)).sum()

    precision = tp/(tp+fp+1e-9)
    recall    = tp/(tp+fn+1e-9)
    f1        = 2*precision*recall/(precision+recall+1e-9)
    acc       = (tp+tn)/(tp+tn+fp+fn+1e-9)
    fpr       = fp/(fp+tn+1e-9)
    return dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))

# ============ 7. 主流程 ============
def main():
    path = "./dynamic_api_call_sequence_per_malware_100_0_306.csv"
    seqs, labels = load_csv_expanded(path)
    train, val, test_benign, test_malware = split_benign_only(seqs, labels)

    # vocab_size 需要从 token 最大值推出来（+1，因为从0计数）
    vocab_size = int(seqs.max()) + 1
    print("vocab_size:", vocab_size)

    window_size = 20
    Xtr, ytr = make_windows(train, window_size)
    Xva, yva = make_windows(val, window_size)

    train_loader = DataLoader(WindowDataset(Xtr, ytr), batch_size=256, shuffle=True)
    val_loader   = DataLoader(WindowDataset(Xva, yva), batch_size=256)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = LSTMLM(vocab_size=vocab_size)
    model = train_model(model, train_loader, val_loader, device=device)

    val_scores = sequence_scores_nll(model, val, window_size, device=device)
    th = pick_threshold(val_scores, q=0.99)
    print("threshold:", th)

    benign_scores  = sequence_scores_nll(model, test_benign, window_size, device=device)
    malware_scores = sequence_scores_nll(model, test_malware, window_size, device=device)
    metrics = evaluate(th, benign_scores, malware_scores)

    print(metrics)

if __name__ == "__main__":
    main()


vocab_size: 307


Epoch 1: 100%|██████████| 236/236 [00:02<00:00, 111.33it/s]


ep1: train=1.9311, val=1.3203


Epoch 2: 100%|██████████| 236/236 [00:01<00:00, 152.61it/s]


ep2: train=1.0468, val=1.0255


...

ep38: train=0.0634, val=1.0226


Epoch 39: 100%|██████████| 236/236 [00:01<00:00, 161.29it/s]


ep39: train=0.0649, val=1.0109


Epoch 40: 100%|██████████| 236/236 [00:01<00:00, 160.91it/s]


ep40: train=0.0666, val=1.0121
threshold: 2.968801293373108
{'acc': np.float64(0.1308876179848391), 'precision': np.float64(0.9990780011062144), 'recall': np.float64(0.1265976587143929), 'f1': np.float64(0.224720032981612), 'fpr': np.float64(0.0230414746542717), 'tp': 5418, 'tn': 212, 'fp': 5, 'fn': 37379}
